In [3]:
pip install -r MP-SENet/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 70.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.6/25.6 MB 52.0 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Installing build dependencies ... error
error: subprocess-exited-with-error

× pip subprocess to install build dependencies did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [8]:
pip install pydub

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#Train model
'''%env CUDA_VISIBLE_DEVICES=0,1
!python MP-SENet/train.py --config MP-SENet/config.json'''

'%env CUDA_VISIBLE_DEVICES=0,1\n!python MP-SENet/train.py --config MP-SENet/config.json'

In [6]:
#Create input test file
import os

#def write_audio_paths_to_file(directory, output_file='/content/drive/MyDrive/TIL_personal/test0.txt'):
def write_audio_paths_to_file(directory, output_file='/content/drive/MyDrive/TIL_personal/advanced/test.txt'):
    with open(output_file, 'w') as file:
        for root, _, files in os.walk(directory):
            for filename in files:
                if filename.lower().endswith(".wav"):
                    full_path = os.path.join(root, filename)
                    file.write(f"{filename.split('.')[0]}|{full_path}\n")

#audio_directory = '/content/drive/MyDrive/TIL_personal/test_audios'
audio_directory = '/content/drive/MyDrive/TIL_personal/test_audios/audio_sliced'
write_audio_paths_to_file(audio_directory)

In [ ]:
# Slices audio into disjoint segments of specified length (in seconds)

import pandas as pd
import torchaudio
import torch
from pathlib import Path
import os

########################## SETTINGS ##########################
segment_length_seconds = 5.
input_csv_path = Path("/content/drive/MyDrive/TIL_personal/advanced/audio/metadata.csv")
sliced_csv_path = Path("/content/drive/MyDrive/TIL_personal/advanced/audio_sliced/sliced_filenames.csv")
input_audio_dir = Path("/content/drive/MyDrive/TIL_personal/advanced/audio")
sliced_output_dir = Path("/content/drive/MyDrive/TIL_personal/advanced/audio_sliced")

os.makedirs(sliced_output_dir, exist_ok=True)

def slice_audio(audio_path, segment_length_seconds):
  wav, sr = torchaudio.load(audio_path)
  segment_length_samples = int(segment_length_seconds * sr)

  slices = list(torch.split(wav, segment_length_samples, dim=1))
  # slices = []
  # curr_start = 0
  # curr_end = min(segment_length_samples, wav.shape[1])
  # while True:
  #   if curr_start >= wav.shape[1]:
  #     break

  #   slices.append(wav[:, curr_start:curr_end])
  #   curr_start = curr_end
  #   curr_end += segment_length_samples

  return slices, sr

def df_map_fn(row):
  slices, sr = slice_audio(input_audio_dir / row['file_name'], segment_length_seconds)

  # save audio slices
  new_filenames = []
  for i, slice in enumerate(slices):
    slice_filename = sliced_output_dir / (os.path.splitext(row["file_name"])[0] + f"-slice-{i+1}.wav")
    new_filenames.append(slice_filename.name)
    torchaudio.save(slice_filename, slice, sr)

  # reassign file_name column for exploding later
  row['file_name'] = new_filenames
  return row

df = pd.read_csv(input_csv_path)
df = df.apply(df_map_fn, axis=1).explode("file_name")
df.to_csv(sliced_csv_path, index=False)

In [7]:
!python /content/MP-SENet/inference.py --checkpoint_file /content/MP-SENet/best_ckpt/g_best \
--input_test_file /content/drive/MyDrive/TIL_personal/advanced/test.txt --input_noisy_wavs_dir /content/drive/MyDrive/TIL_personal/test_audios/audio_sliced \
--output_dir /content/drive/MyDrive/TIL_personal/advanced/generated_files

Streaming output truncated to the last 5000 lines.
audio_2501-slice-2
audio_2502-slice-1
audio_2502-slice-2
audio_2503-slice-1
audio_2503-slice-2
audio_2504-slice-1
audio_2504-slice-2
audio_2505-slice-1
audio_2505-slice-2
audio_2506-slice-1
audio_2506-slice-2
audio_2506-slice-3
audio_2506-slice-4
audio_2506-slice-5
audio_2507-slice-1
audio_2507-slice-2
audio_2508-slice-1
audio_2508-slice-2
audio_2509-slice-1
audio_2510-slice-1
audio_2510-slice-2
audio_2511-slice-1
audio_2511-slice-2
audio_2512-slice-1
audio_2513-slice-1
audio_2513-slice-2
audio_2513-slice-3
audio_2514-slice-1
audio_2514-slice-2
audio_2515-slice-1
audio_2515-slice-2
audio_2516-slice-1
audio_2516-slice-2
audio_2517-slice-1
audio_2517-slice-2
audio_2517-slice-3
audio_2518-slice-1
audio_2519-slice-1
audio_2519-slice-2
audio_2520-slice-1
audio_2520-slice-2
audio_2520-slice-3
audio_2521-slice-1
audio_2521-slice-2
audio_2522-slice-1
audio_2522-slice-2
audio_2523-slice-1
audio_2523-slice-2
audio_2524-slice-1
audio_2524-slice-2

In [14]:
from pydub import AudioSegment
from collections import defaultdict

def join_wav_files(audio, input_list, output_path):
    """
    Join two WAV files together and save the result as a new WAV file.
    """
    input_list.sort()
    combined_audio = AudioSegment.from_wav("/content/drive/MyDrive/TIL_personal/advanced/generated_files/"\
                                           +audio+"-slice-"+input_list[0]+".wav")
    for i in range(1, len(input_list)):
      combined_audio += AudioSegment.from_wav("/content/drive/MyDrive/TIL_personal/advanced/generated_files/"\
                                           +audio+"-slice-"+input_list[i]+".wav")

    # Export the combined audio to a new file
    combined_audio.export(output_path+audio+".wav", format='wav')

combined_output_dir = "/content/drive/MyDrive/TIL_personal/advanced/denoised_files/"
os.makedirs(combined_output_dir, exist_ok=True)
audiodict = defaultdict(list)
with open("/content/drive/MyDrive/TIL_personal/advanced/test.txt") as f:
  for line in f:
    line = line.split("-slice-")
    audiodict[line[0]].append(line[1][0])

# Join the two WAV files
for audio, part in audiodict.items():
  join_wav_files(audio, part, combined_output_dir)